# Data Quality Starter

### Case 2: Detect possible alternative names
When you are trying to deduplicate leads entered on our website, is important to understand possible nickname variations that a person could use rather than using their given name.

In [ ]:
ALTER SESSION SET QUERY_TAG = '{
    "origin": "sf_sit-is", 
    "name": "marketing_data_foundation_starter_spcs", 
    "version": {"major": 3, "minor": 0},
    "attributes": {"is_quickstart": 0, "source": "notebook"}
}';


select first_name, last_name,
 snowflake.cortex.complete('mixtral-8x7b'
        , [
        {'role': 'system', 'content': 'You are a marketing expert working to deduplicate leads entered on our website. To that end, it is useful to understand possible nickname variations.'
            || 'For example, a person filling out the form could use a nickname, rather than their given name. Or they could enter their given name when they commonly use a nickname. '
            || 'Therefore, please identify possible name variations that the given person may use. Do not include the last name in any variation. Do not include the original name. Do not include alternate spellings.'
            || 'Return 5 or fewer possible names separated with commas with an xml tag <response>.'
            || 'You should provide only the list of names and no text other than the xml tag, in the form <response>name1, name2, name3</response>.' },
        {'role': 'user', 'content': '<data>'
            || 'Name: '
            || first_name || ' ' || last_name
            || '</data>'}
            ]
        , {'max_tokens': 35}
        ):choices[0]:messages::string as spam_filter_raw
    , regexp_substr(spam_filter_raw, '<response>(.*?)<\/response>', 1, 1, 'e', 1) as spam_filter
from LLM_DEMO.DEMO.customer_information;